In [1]:
!pip install nltk spacy transformers --force-reinstall numpy==1.24.4 scipy==1.10.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.0/68.0 kB 4.7 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of thinc to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 96.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.1/34.1 MB 54.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 62.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.0/33.0 MB 57.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 55.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━

# Import libraries

In [2]:
import pandas as pd
import numpy as np
import spacy
import re
from bs4 import BeautifulSoup

import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import warnings

# warnings.filterwarnings("ignore")

# Load model

In [3]:
# Load spaCy model
nlp = spacy.load("en_core_web_sm")

# Define utils & functions

In [4]:
# Define contractions dictionary
contractions = {
    "aren't": "are not",
    "can't": "cannot",
    "couldn't": "could not",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he would",
    "he'll": "he will",
    "he's": "he is",
    "i'd": "i would",
    "i'll": "i will",
    "i'm": "i am",
    "i've": "i have",
    "isn't": "is not",
    "let's": "let us",
    "might've": "might have",
    "mustn't": "must not",
    "shan't": "shall not",
    "she'd": "she would",
    "she'll": "she will",
    "she's": "she is",
    "shouldn't": "should not",
    "that's": "that is",
    "there's": "there is",
    "they'd": "they would",
    "they'll": "they will",
    "they're": "they are",
    "they've": "they have",
    "we'd": "we would",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "where's": "where is",
    "who'd": "who would",
    "who'll": "who will",
    "who're": "who are",
    "who's": "who is",
    "who've": "who have",
    "won't": "will not",
    "wouldn't": "would not",
    "you'd": "you would",
    "you'll": "you will",
    "you're": "you are",
    "you've": "you have"
}

In [5]:
# Function to expand contractions in text
def expand_contractions(text):
    for contraction, expanded in contractions.items():
        text = re.sub(r'\b' + contraction + r'\b', expanded, text, flags=re.IGNORECASE)
    return text

# Function to strip HTML tags
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

# Function to remove special characters and optionally digits
def remove_special_letters(text, remove_digits=True):
    pattern = r'[^a-zA-Z\s]' if remove_digits else r'[^a-zA-Z0-9\s]'
    text = re.sub(pattern, '', text)
    return text

# Function to preprocess a batch of texts
def preprocess_text_batch(texts):
    texts = [expand_contractions(str(t)) for t in texts]
    texts = [strip_html(t) for t in texts]
    texts = [remove_special_letters(t, remove_digits=True).lower() for t in texts]
    
    # Load NLTK stopwords
    stop_words = set(stopwords.words('english'))
    
    # Tokenization and Lemmatization using spaCy and NLTK
    lemmatizer = WordNetLemmatizer()
    tokens = []
    for doc in nlp.pipe(texts, batch_size=1000):
        tokenized = []
        for token in doc:
            if token.text not in stop_words:
                pos = get_wordnet_pos(token.pos_)
                if pos:
                    lemma = lemmatizer.lemmatize(token.text, pos=pos)
                else:
                    lemma = lemmatizer.lemmatize(token.text)
                tokenized.append(lemma)
        tokens.append(tokenized)
    return tokens

# Function to map spaCy POS tags to WordNet POS tags
def get_wordnet_pos(spacy_pos):
    from nltk.corpus import wordnet
    if spacy_pos in ['ADJ']:
        return wordnet.ADJ
    elif spacy_pos in ['VERB']:
        return wordnet.VERB
    elif spacy_pos in ['NOUN']:
        return wordnet.NOUN
    elif spacy_pos in ['ADV']:
        return wordnet.ADV
    return None

# Load raw data

In [6]:
# Load dataset
imdb_data = pd.read_csv('/kaggle/input/imdb-datasetin/IMDB Dataset(in).csv')
imdb_data['original_review'] = imdb_data['review']

# Apply preprocessing

In [7]:
# Apply the batch preprocessing function to the raw reviews
print("Preprocessing data...")
texts = imdb_data['original_review'].tolist()
preprocessed_texts = preprocess_text_batch(texts)
imdb_data['review'] = preprocessed_texts
print("Preprocessing complete!")

Preprocessing data...
Preprocessing complete!


In [8]:
# Save the preprocessed results
output_df = pd.DataFrame({
    'sentiment': imdb_data['sentiment'],
    'preprocessed_tokens': [" ".join(tokens) if isinstance(tokens, list) else "" for tokens in preprocessed_texts]
})
output_file = '/kaggle/working/preprocessed_results.csv'
output_df.to_csv(output_file, index=False)

In [9]:
# Print sample output
print("\nPreview of the output DataFrame:")
print(output_df.head(5))


Preview of the output DataFrame:
  sentiment                                preprocessed_tokens
0  positive  one reviewer mention watch   oz episode hook r...
1  positive  wonderful little production filming technique ...
2  positive  think wonderful way spend time hot summer week...
3  negative  basically family little boy jake think zombie ...
4  positive  petter matteis love time money visually stunni...
